### Clustering of neighborhoods in Toronto by Eliana Salvemini ###

In [19]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt # plotting library

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import wikipedia
import requests
#https://beautiful-soup-4.readthedocs.io/en/latest/
from bs4 import BeautifulSoup   #Python package for parsing HTML and XML documents
import time
print('Libraries imported.')

Libraries imported.


In [20]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url)

I used the beautiful soup library to scrub the wikipedia table page

In [21]:
#turn the HTML into a beautiful soup text object
b = BeautifulSoup(html.text, 'lxml')
print(b.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":861324217,"wgRevisionId":861324217,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

Create empty arrays to collect postcode, borough and neighborhood

In [22]:
postcode = []
borough = []
neighbourhood = []

Loop for each row of the table: 
    - skip the row with empty borough
    - replace the neighbourhood with the borough for row with missing neighbourhood
    - fill the postcode, the borough and the neighbourhood for right row

In [23]:
# get the table on the page
for table in b.find_all(name='table', class_='wikitable sortable'):
    for j in table.find_all(name='tr')[1:]:
        cells = j.findAll(["th", "td"])
        # if borough is missing skip the row 
        if ("Not assigned" in cells[1].text.replace('\n','').replace("\'","")):
            print("Not Assigned borough Found, skip the row", cells[0].text)
        # if the neighbourhood is missin than replaces with the borough
        elif ("Not assigned" in cells[2].text.replace('\n','').replace("\'","")):
            print("Not Assigned neighbourhood Found, replace with borough", cells[1].text)
            print("postcode ", cells[0].text)
            postcode.append(cells[0].text)
            print("borough ", cells[1].text)
            borough.append(cells[1].text.replace('\n','').replace("\'",""))
            print("neighbourhood ", cells[1].text)
            neighbourhood.append(cells[1].text.replace('\n','').replace("\'",""))
        else:
            #this is the with row
            print("postcode ", cells[0].text)
            postcode.append(cells[0].text)
            print("borough ", cells[1].text)
            borough.append(cells[1].text.replace('\n','').replace("\'",""))
            print("neighbourhood ", cells[2].text)
            neighbourhood.append(cells[2].text.replace('\n','').replace("\'",""))

time.sleep(1)

Not Assigned borough Found, skip the row M1A
Not Assigned borough Found, skip the row M2A
postcode  M3A
borough  North York
neighbourhood  Parkwoods

postcode  M4A
borough  North York
neighbourhood  Victoria Village

postcode  M5A
borough  Downtown Toronto
neighbourhood  Harbourfront

postcode  M5A
borough  Downtown Toronto
neighbourhood  Regent Park

postcode  M6A
borough  North York
neighbourhood  Lawrence Heights

postcode  M6A
borough  North York
neighbourhood  Lawrence Manor

Not Assigned neighbourhood Found, replace with borough Queen's Park
postcode  M7A
borough  Queen's Park
neighbourhood  Queen's Park
Not Assigned borough Found, skip the row M8A
postcode  M9A
borough  Etobicoke
neighbourhood  Islington Avenue

postcode  M1B
borough  Scarborough
neighbourhood  Rouge

postcode  M1B
borough  Scarborough
neighbourhood  Malvern

Not Assigned borough Found, skip the row M2B
postcode  M3B
borough  North York
neighbourhood  Don Mills North

postcode  M4B
borough  East York
neighbourho

Check that the lenght of the array are the same

In [24]:
print('The are {} postcode'.format(len(postcode)))
print('POSTCODE: \n\n', postcode[1:30])
print('The are {} borough'.format(len(borough)))
print('BOROUGH: \n\n', borough[1:30])
print('The are {} neighbourhood'.format(len(neighbourhood)))
print('NEIGHBOURHOOD: \n\n', neighbourhood[1:30])

The are 212 postcode
POSTCODE: 

 ['M4A', 'M5A', 'M5A', 'M6A', 'M6A', 'M7A', 'M9A', 'M1B', 'M1B', 'M3B', 'M4B', 'M4B', 'M5B', 'M5B', 'M6B', 'M9B', 'M9B', 'M9B', 'M9B', 'M9B', 'M1C', 'M1C', 'M1C', 'M3C', 'M3C', 'M4C', 'M5C', 'M6C', 'M9C']
The are 212 borough
BOROUGH: 

 ['North York', 'Downtown Toronto', 'Downtown Toronto', 'North York', 'North York', 'Queens Park', 'Etobicoke', 'Scarborough', 'Scarborough', 'North York', 'East York', 'East York', 'Downtown Toronto', 'Downtown Toronto', 'North York', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Etobicoke', 'Scarborough', 'Scarborough', 'Scarborough', 'North York', 'North York', 'East York', 'Downtown Toronto', 'York', 'Etobicoke']
The are 212 neighbourhood
NEIGHBOURHOOD: 

 ['Victoria Village', 'Harbourfront', 'Regent Park', 'Lawrence Heights', 'Lawrence Manor', 'Queens Park', 'Islington Avenue', 'Rouge', 'Malvern', 'Don Mills North', 'Woodbine Gardens', 'Parkview Hill', 'Ryerson', 'Garden District', 'Glencairn', 'Cloverdale', '

define the dataframe columns

In [25]:
column_names = ['PostalCode', 'Borough', 'Neighborhood']

Create an empty dataframe with the new columns

In [26]:
#create a dictionary with data
dictionary = {'PostalCode':postcode,'Borough':borough,'Neighborhood':neighbourhood}

# instantiate the empty dataframe
neighborhoodsToronto = pd.DataFrame(dictionary, columns=column_names)
print(neighborhoodsToronto.head(10))

  PostalCode           Borough      Neighborhood
0        M3A        North York         Parkwoods
1        M4A        North York  Victoria Village
2        M5A  Downtown Toronto      Harbourfront
3        M5A  Downtown Toronto       Regent Park
4        M6A        North York  Lawrence Heights
5        M6A        North York    Lawrence Manor
6        M7A       Queens Park       Queens Park
7        M9A         Etobicoke  Islington Avenue
8        M1B       Scarborough             Rouge
9        M1B       Scarborough           Malvern


Use lambda function to collect the row with the same PostalCode

In [27]:
#aggregate the postalcode according to the format required
df = neighborhoodsToronto.groupby('PostalCode').agg(lambda x: ', '.join(set(x))).reset_index()
print(df.head(20))

   PostalCode      Borough                                       Neighborhood
0         M1B  Scarborough                                     Malvern, Rouge
1         M1C  Scarborough             Port Union, Highland Creek, Rouge Hill
2         M1E  Scarborough                  West Hill, Guildwood, Morningside
3         M1G  Scarborough                                             Woburn
4         M1H  Scarborough                                          Cedarbrae
5         M1J  Scarborough                                Scarborough Village
6         M1K  Scarborough        East Birchmount Park, Kennedy Park, Ionview
7         M1L  Scarborough                    Golden Mile, Clairlea, Oakridge
8         M1M  Scarborough    Scarborough Village West, Cliffside, Cliffcrest
9         M1N  Scarborough                        Cliffside West, Birch Cliff
10        M1P  Scarborough  Wexford Heights, Scarborough Town Centre, Dors...
11        M1R  Scarborough                                  Wexf

print the shape of the dataframe

In [28]:
print('Shape of the dataframe: ', neighborhoodsToronto.shape)

Shape of the dataframe:  (212, 3)


In [29]:
#check for some particular values to verify the correctness of the solution
print('PostalCode M9V \n', df.loc[df['PostalCode']=='M9V'])

print('PostalCode M8A \n', df.loc[df['PostalCode']=='M8A'])

print('PostalCode M7A \n',df.loc[df['PostalCode']=='M7A'])

PostalCode M9V 
     PostalCode    Borough                                       Neighborhood
101        M9V  Etobicoke  Beaumond Heights, Albion Gardens, Jamestown, T...
PostalCode M8A 
 Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []
PostalCode M7A 
    PostalCode      Borough Neighborhood
85        M7A  Queens Park  Queens Park



### Foursquare API to get location data

In [30]:
#Initialize the variables 
lat_lng_coords = None
latitude = []
longitude = []

In [ ]:
# loop until get the coordinates
for i in df['PostalCode']:
    print("postcode ", i)
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(i))
        lat_lng_coords = g.latlng

    latitude.append(lat_lng_coords[0])
    longitude.append(lat_lng_coords[1])

postcode  M1B


Add these 2 new columns to the dataframe

In [58]:
df['Latitude'] = latitude
df['Longitude'] = longitude
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Port Union, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, Guildwood]], West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview, Kennedy Park, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge, Golden Mile, Clairlea",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West, Cliffcrest, Cliffside",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


In [28]:
df.to_csv('example.csv', sep='\t')

In [35]:
df.head(20)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


Sort the dataframe according to the values of 'PostalCode' column

In [34]:
dfSorted = df.sort_values(['PostalCode'],  ascending=True)
dfSorted.head(20)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill"
2,M1E,Scarborough,"West Hill, Guildwood, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Kennedy Park, Ionview"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Scarborough Village West, Cliffside, Cliffcrest"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


I am having some troubles with the geocoder.google API so I use the csv file to get the coordinate

In [31]:
#read the csv file from local file system
coordinates = pd.read_csv('C:/Workspace/Python/data/Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Sort the 'Postal Code' column just to be sure

In [32]:
coordinates = coordinates.sort_values(['Postal Code'],  ascending=True)
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [33]:
coordinates.shape

(103, 3)

In [35]:
dfSorted.shape

(103, 3)

Add the latitude and the longitude columns

In [36]:
dfSorted['Latitude'] = coordinates['Latitude']
dfSorted['Longitude'] = coordinates['Longitude']
dfSorted.head(20)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Highland Creek, Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"West Hill, Guildwood, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Kennedy Park, Ionview",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West, Cliffside, Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


Define Foursquare Credentials and Version

In [37]:
CLIENT_ID = '1RHG0VKNFIUBDXQGDKDQEZP1WJXPIYBKMGIQO1Z0RYGZRM1A' # your Foursquare ID
CLIENT_SECRET = 'VJ4E4WB2B1Z4AEGNAHJYQQCJGPERRA5RE3K15FLY02R0HFYQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: 1RHG0VKNFIUBDXQGDKDQEZP1WJXPIYBKMGIQO1Z0RYGZRM1A
CLIENT_SECRET:VJ4E4WB2B1Z4AEGNAHJYQQCJGPERRA5RE3K15FLY02R0HFYQ


Get the geografical coordinate of Toronto city

In [38]:
address = 'Toronto, Ontario'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitudeTo = location.latitude
longitudeTo = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitudeTo, longitudeTo))

C:\ProgramData\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:138: UserWarning: Using Nominatim with the default "geopy/1.16.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


Let's get the top 100 venues that are in Toronto

In [39]:
search_query = ''
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitudeTo, longitudeTo, VERSION, search_query, radius, LIMIT)
print(url)


https://api.foursquare.com/v2/venues/explore?client_id=1RHG0VKNFIUBDXQGDKDQEZP1WJXPIYBKMGIQO1Z0RYGZRM1A&client_secret=VJ4E4WB2B1Z4AEGNAHJYQQCJGPERRA5RE3K15FLY02R0HFYQ&ll=43.653963,-79.387207&v=20180605&query=&radius=500&limit=100


In [40]:
#Send the GET request and examine the results
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5bb0ea0c351e3d4c935e992d'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5227bb01498e17bf485e6202-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_',
          'suffix': '.png'},
         'id': '4f2a25ac4b909258e854f55f',
         'name': 'Neighborhood',
         'pluralName': 'Neighborhoods',
         'primary': True,
         'shortName': 'Neighborhood'}],
       'id': '5227bb01498e17bf485e6202',
       'location': {'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 174,
        'formattedAddress': ['Toronto ON', 'Canada'],
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng': -79.38529600606677}],
        'lat': 43.6532

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']

    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [42]:
#clean the json and structure it into a pandas dataframe
venues = results['response']['groups'][0]['items']

print("VENUES \n", venues)


VENUES 
 [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '5227bb01498e17bf485e6202', 'name': 'Downtown Toronto', 'location': {'lat': 43.65323167517444, 'lng': -79.38529600606677, 'labeledLatLngs': [{'label': 'display', 'lat': 43.65323167517444, 'lng': -79.38529600606677}], 'distance': 174, 'cc': 'CA', 'city': 'Toronto', 'state': 'ON', 'country': 'Canada', 'formattedAddress': ['Toronto ON', 'Canada']}, 'categories': [{'id': '4f2a25ac4b909258e854f55f', 'name': 'Neighborhood', 'pluralName': 'Neighborhoods', 'shortName': 'Neighborhood', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/neighborhood_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-5227bb01498e17bf485e6202-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue':

Flatten JSON into a dataframe

In [43]:
nearby_venues = json_normalize(venues)
nearby_venues.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,...,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.neighborhood,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups,venue.venuePage.id
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-5227bb01498e17bf485e6202-0,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",5227bb01498e17bf485e6202,NaN,CA,Toronto,Canada,NaN,...,"[{'label': 'display', 'lat': 43.65323167517444...",43.653232,-79.385296,NaN,NaN,ON,Downtown Toronto,0,[],NaN
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ad4c05ef964a520e2f620e3-1,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",4ad4c05ef964a520e2f620e3,55 Centre Avenue,CA,Toronto,Canada,University Ave. and Dundas St W.,...,"[{'label': 'display', 'lat': 43.65439630500274...",43.654396,-79.386500,NaN,M5G 2H5,ON,Textile Museum of Canada,0,[],56305293
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4ae7b27df964a52068ad21e3-2,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",4ae7b27df964a52068ad21e3,122 Elizabeth St.,CA,Toronto,Canada,at Dundas St. W,...,"[{'label': 'display', 'lat': 43.65526771691681...",43.655268,-79.385165,NaN,M5G 1P5,ON,Japango,0,[],NaN
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-504bbf2ce4b0168121235cbe-3,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",504bbf2ce4b0168121235cbe,179 Dundas St. W,CA,Toronto,Canada,btwn Centre Ave. & Chestnut St.,...,"[{'label': 'display', 'lat': 43.65515746756124...",43.655157,-79.386501,NaN,M5G 1Z8,ON,Sansotei Ramen 三草亭,0,[],NaN
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4f513029e4b07c3382c9fdb9-4,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",4f513029e4b07c3382c9fdb9,250 Dundas Street West,CA,Toronto,Canada,Simcoe Street,...,"[{'label': 'display', 'lat': 43.65457125894357...",43.654571,-79.389450,NaN,M5T 2Z5,ON,Cafe Plenty,0,[],NaN


Consider only the columns 'venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng'

In [44]:
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Downtown Toronto,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.653232,-79.385296
1,Textile Museum of Canada,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",43.654396,-79.386500
2,Japango,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",43.655268,-79.385165
3,Sansotei Ramen 三草亭,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",43.655157,-79.386501
4,Cafe Plenty,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",43.654571,-79.389450


Filter the category for each row

In [45]:
nearby_venues['categories'] = nearby_venues.apply(get_category_type, axis=1)


In [46]:
nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng,categories
0,Downtown Toronto,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.653232,-79.385296,Neighborhood
1,Textile Museum of Canada,"[{'id': '4bf58dd8d48988d18f941735', 'name': 'A...",43.654396,-79.386500,Art Museum
2,Japango,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",43.655268,-79.385165,Sushi Restaurant
3,Sansotei Ramen 三草亭,"[{'id': '55a59bace4b013909087cb24', 'name': 'R...",43.655157,-79.386501,Ramen Restaurant
4,Cafe Plenty,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",43.654571,-79.389450,Café


Clean columns (remove the . from the

In [47]:
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

In [48]:
nearby_venues.columns

Index(['name', 'categories', 'lat', 'lng', 'categories'], dtype='object')

nearby_venues.head()

How many venues were returned by Foursquare?

In [50]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

69 venues were returned by Foursquare.


# 2. Explore Neighborhoods in Toronto

Get all Toronto boroughs data

In [93]:
toronto_data = dfSorted[dfSorted['Borough'].str.contains("Toronto")]

In [94]:
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Check the boroughs filtered

In [95]:
toronto_data['Borough'].unique()

array(['East Toronto', 'Central Toronto', 'Downtown Toronto',
       'West Toronto'], dtype=object)

In [104]:
len(toronto_data['Neighborhood'].unique())

38

I repeated the same function to repeat the same process to all the neighborhoods in Toronto

In [96]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):

    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            VERSION,
            lat,
            lng,
            radius,
            LIMIT)

        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']

    return(nearby_venues)

Run the above function on each neighborhood and create a new dataframe called *toronto_venues*

In [99]:
toronto_venues = getNearbyVenues(toronto_data['Neighborhood'], toronto_data['Latitude'], toronto_data['Longitude'])
toronto_venues.head()

The Beaches
Riverdale, The Danforth West
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Summerhill East, Moore Park
Forest Hill SE, Summerhill West, Rathnelly, Deer Park, South Hill
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Regent Park, Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
King, Adelaide, Richmond
Union Station, Toronto Islands, Harbourfront East
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
Yorkville, North Midtown, The Annex
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
Railway Lands, King and Spadina, South Niagara, CN Tower, Harbourfront West, Island airport, Bathurst Quay
Stn A PO Boxes 25 The Esplanade
Underground city, First Canadian Place
Christie
Dufferin, Dovercourt Village
Trinity, Little Portugal
Parkdale Village, Brockton, Exhibition Place
High Park, The 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"Riverdale, The Danforth West",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


Let's check the size of the resulting dataframe

In [100]:
print(toronto_venues.shape)

(1697, 7)


In [107]:
len(toronto_venues.sort_values(['Neighborhood'])['Neighborhood'].unique())

38

Let's check how many venues were returned for each neighborhood

In [108]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,54,54,54,54,54,54
Business reply mail Processing Centre969 Eastern,18,18,18,18,18,18
"Cabbagetown, St. James Town",48,48,48,48,48,48
Central Bay Street,83,83,83,83,83,83
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16
Church and Wellesley,89,89,89,89,89,89
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37


Let's find out how many unique categories can be curated from all the returned venues

In [109]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


# Analyze Each Neighborhood

In [110]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [111]:
toronto_onehot['Neighborhood'].head()

0    0
1    0
2    0
3    1
4    0
Name: Neighborhood, dtype: uint8

I drop the neighborhood column since it is not important

In [112]:
toronto_onehot.drop(columns=['Neighborhood'], axis = 1, inplace=True)
toronto_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Add neighborhood column back to dataframe

In [113]:
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

In [114]:
len(list(toronto_onehot.columns.values))

232

In [115]:
toronto_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio,Neighborhood
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The Beaches
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The Beaches
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The Beaches
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,The Beaches
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"Riverdale, The Danforth West"


Move neighborhood column to the first column

In [116]:
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [117]:
toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Riverdale, The Danforth West",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Let's examine the new dataframe size

In [118]:
print("Shape of dataset toronto_onehot: ", toronto_onehot.shape)

Shape of dataset toronto_onehot:  (1697, 232)


Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [119]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000
1,Business reply mail Processing Centre969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.055556
2,"Cabbagetown, St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,...,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000
3,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012048,...,0.00,0.0,0.0,0.012048,0.0,0.00,0.0,0.012048,0.0,0.012048
4,"Chinatown, Grange Park, Kensington Market",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.01,0.0,0.0,0.060000,0.0,0.04,0.0,0.010000,0.0,0.000000


Let's print each neighborhood along with the top 5 most common venues

In [120]:
num_top_venues = 5

In [121]:
for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.06
2    Restaurant  0.04
3   Cheese Shop  0.04
4      Beer Bar  0.04


----Business reply mail Processing Centre969 Eastern----
         venue  freq
0  Yoga Studio  0.06
1   Restaurant  0.06
2   Smoke Shop  0.06
3          Spa  0.06
4   Skate Park  0.06


----Cabbagetown, St. James Town----
               venue  freq
0        Coffee Shop  0.08
1         Restaurant  0.08
2        Pizza Place  0.06
3  Indian Restaurant  0.04
4               Café  0.04


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.16
1                 Café  0.06
2   Italian Restaurant  0.05
3  Japanese Restaurant  0.04
4       Sandwich Place  0.04


----Chinatown, Grange Park, Kensington Market----
                           venue  freq
0                           Café  0.07
1                            Bar  0.06
2  Vegetarian / Vegan Restaurant  0.06
3             Chinese Restaurant  0.05
4            

Let's write a function to sort the venues in descending order

In [125]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)

    return row_categories_sorted.index.values[0:num_top_venues]

Let's create the new dataframe and display the top 10 venues for each neighborhood

In [126]:
num_top_venues = 10

In [127]:
indicators = ['st', 'nd', 'rd']

Create columns according to number of top venues

In [129]:
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

Create a new dataframe

In [130]:
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)

In [131]:
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

In [132]:
neighborhoods_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Business reply mail Processing Centre969 Eastern,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Cabbagetown, St. James Town",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Central Bay Street,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Chinatown, Grange Park, Kensington Market",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Christie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Church and Wellesley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Commerce Court, Victoria Hotel",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Davisville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Davisville North,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [133]:
for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)


In [134]:
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Restaurant,Cheese Shop,Café
1,Business reply mail Processing Centre969 Eastern,Yoga Studio,Auto Workshop,Park,Pizza Place,Recording Studio,Butcher,Restaurant,Burrito Place,Brewery,Skate Park
2,"Cabbagetown, St. James Town",Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Café,Indian Restaurant,Market,Pub,Bakery,Park
3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Bar,Bubble Tea Shop,Burger Joint,Sandwich Place,Salad Place,Indian Restaurant
4,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bakery,Dumpling Restaurant,Coffee Shop,Hotpot Restaurant


# Cluster Neighborhoods 

Run k-means to cluster the neighborhood into 5 clusters

In [135]:
# set number of clusters
kclusters = 5

Drop the column Neighborhood since kmeans can run only on numerical data

In [136]:
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

In [137]:
toronto_grouped_clustering.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.055556
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.020833,0.0,...,0.00,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.012048,0.0,...,0.00,0.0,0.0,0.012048,0.0,0.00,0.0,0.012048,0.0,0.012048
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.01,0.0,0.0,0.060000,0.0,0.04,0.0,0.010000,0.0,0.000000


Run k-means clustering

In [138]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

Check cluster labels generated for each row in the dataframe

In [139]:
len(kmeans.labels_)

38

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [140]:
toronto_merged = toronto_data

Add clustering labels

In [141]:
toronto_merged['Cluster Labels'] = kmeans.labels_

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [142]:
print("toronto_merged \n")
toronto_merged.head()

toronto_merged 



,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0
41,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,0
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0


In [143]:
print("neighborhoods_venues_sorted \n")
neighborhoods_venues_sorted.head()

neighborhoods_venues_sorted 



,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Restaurant,Cheese Shop,Café
1,Business reply mail Processing Centre969 Eastern,Yoga Studio,Auto Workshop,Park,Pizza Place,Recording Studio,Butcher,Restaurant,Burrito Place,Brewery,Skate Park
2,"Cabbagetown, St. James Town",Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Café,Indian Restaurant,Market,Pub,Bakery,Park
3,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Bar,Bubble Tea Shop,Burger Joint,Sandwich Place,Salad Place,Indian Restaurant
4,"Chinatown, Grange Park, Kensington Market",Café,Bar,Vegetarian / Vegan Restaurant,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bakery,Dumpling Restaurant,Coffee Shop,Hotpot Restaurant


Merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood

In [144]:
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [145]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Other Great Outdoors,Pub,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
41,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Yoga Studio,Bubble Tea Shop,Bakery,Spa,Juice Bar
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,0,Pet Store,Ice Cream Shop,Movie Theater,Pub,Burrito Place,Burger Joint,Sandwich Place,Brewery,Liquor Store,Fast Food Restaurant
43,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Italian Restaurant,Bakery,Gastropub,American Restaurant,Fish Market,Juice Bar,New American Restaurant,Latin American Restaurant
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


Finally, let's visualize the resulting clusters

In [146]:
# create map
map_clusters = folium.Map(location=[latitudeTo, longitudeTo], zoom_start=11)

Set color scheme for the clusters

In [147]:
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print("rainbow \n", rainbow)

rainbow 
 ['#8000ff', '#00b5eb', '#80ffb4', '#ffb360', '#ff0000']


Add markers to the map

In [148]:
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

Show the map

In [149]:
map_clusters

# Examine Clusters 

In [150]:
######## CLUSTER 1 ########
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Coffee Shop,Other Great Outdoors,Pub,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
41,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Yoga Studio,Bubble Tea Shop,Bakery,Spa,Juice Bar
42,East Toronto,0,Pet Store,Ice Cream Shop,Movie Theater,Pub,Burrito Place,Burger Joint,Sandwich Place,Brewery,Liquor Store,Fast Food Restaurant
43,East Toronto,0,Café,Coffee Shop,Italian Restaurant,Bakery,Gastropub,American Restaurant,Fish Market,Juice Bar,New American Restaurant,Latin American Restaurant
44,Central Toronto,0,Bus Line,Park,Swim School,Dim Sum Restaurant,Yoga Studio,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
45,Central Toronto,0,Hotel,Grocery Store,Park,Sandwich Place,Breakfast Spot,Burger Joint,Food & Drink Shop,Yoga Studio,Dumpling Restaurant,Doner Restaurant
46,Central Toronto,0,Coffee Shop,Sporting Goods Shop,Italian Restaurant,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Cosmetics Shop,Bagel Shop,Clothing Store
47,Central Toronto,0,Coffee Shop,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Pharmacy,Seafood Restaurant,Café,Sushi Restaurant,Gourmet Shop
48,Central Toronto,0,Restaurant,Playground,Intersection,Trail,Gym,Event Space,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
49,Central Toronto,0,Coffee Shop,Pub,Sandwich Place,American Restaurant,Supermarket,Sushi Restaurant,Light Rail Station,Fried Chicken Joint,Sports Bar,Medical Center


In [151]:
######## CLUSTER 2 ########
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,Central Toronto,1,Café,Sandwich Place,Coffee Shop,Park,Pizza Place,French Restaurant,BBQ Joint,Indian Restaurant,Flower Shop,Pub


In [152]:
######## CLUSTER 3 ########
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
51,Downtown Toronto,2,Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Café,Indian Restaurant,Market,Pub,Bakery,Park
64,Central Toronto,2,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


In [153]:
######## CLUSTER 4 ########
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Downtown Toronto,3,Grocery Store,Café,Park,Athletics & Sports,Coffee Shop,Nightclub,Restaurant,Diner,Baby Store,Italian Restaurant


In [154]:
######## CLUSTER 5 ########
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
56,Downtown Toronto,4,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Steakhouse,Seafood Restaurant,Farmers Market,Restaurant,Cheese Shop,Café
